In [4]:
import json
from tqdm import tqdm, trange

# Conver jsonl to json file

In [2]:




import json
import re

input_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/aime2025-I.jsonl"
output_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/aime2025.json"

def extract_last_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", r"\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"\\boxed\{(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) == 0:
        res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        num_str = res[-1][0]
        return float(num_str)
    else:
        return 0.0
    
def extract_last_query(text):
    # 找到所有### Query: 后的内容，直到遇到下一个###或文本结尾
    matches = re.findall(r'### Query:\s*(.*?)(?:\n###|\Z)', text, re.DOTALL)
    if matches:
        return matches[-1].strip()  # 取最后一个
    return None

def extract_last_response(text):
    # 找到所有### Query: 后的内容，直到遇到下一个###或文本结尾
    matches = re.findall(r'### Response:\s*(.*?)(?:\n###|\Z)', text, re.DOTALL)
    if matches:
        return matches[-1].strip()  # 取最后一个
    return None

results = []
with open(input_path, "r") as f:
    for line in f:
        line = json.loads(line)
        line["query"] = line["question"]
        line['answer'] = extract_last_num(line['answer'])
        del line["question"]
        # line["query"] = extract_last_query(line["prompt"]).strip()
        # line["answer"] = extract_last_num(extract_last_response(line["prompt"]))
        # line["generated_texts"] = line["model_output"]
        # del line["model_output"]
        results.append(line)
        
with open(output_path, "w") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
    
print(f"{len(results)} lines have been processed and saved to {output_path}.")

30 lines have been processed and saved to /home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/aime2025.json.


In [19]:
print(results[0]["query"], results[0]["answer"])

Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook. 4.0


In [2]:
results[0]

{'idx': '0_0',
 'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.',
 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.',
 'response': '### Template:\nFind the number of people who crossed the finish line faster than <person_name> if <person_name> is in <position> place.\n\n### Python Code:\n```python\n# Variable definitions\nperson_name = "Jungkook"\nposition = 5\n\n# Calculation\npeople_faster = position - 1\n\n# Output\nprint(people_faster)\n```',
 'template': 'Find the number of people who crossed the finish line faster than <person_name> if <person_name> is in <position> place.',
 'python': '# Variable definitions\nperson_name = "Jungkook"\nposition = 5\n\n# Calculation\npeople_faster = position - 1\n\n# Output\nprint(people_faster)',
 'new_query': 'Find the number of people who crossed the finish line faster than "Jungkook" if "Jungkook" is in 9 place.',
 'new_code': '# V

# Convert Parquet to json file

In [6]:
import pandas as pd
import json
import re

# 读取 Parquet 文件
parquet_file_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/aime_2024_problems.parquet"  # 替换为你的 Parquet 文件路径
json_file_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/aime2024.json"  # 保存的 JSON 文件路径


def extract_last_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", r"\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"\\boxed\{(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) == 0:
        res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        num_str = res[-1][0]
        return float(num_str)
    else:
        return 0.0

df = pd.read_parquet(parquet_file_path)

prompt = """### Instruction:
{}

### Response: 
Let’s think step by step."""

# 将 DataFrame 转换为 JSON 格式
json_data = df.to_json(orient="records", indent=4)
json_data = json.loads(json_data)

data = []

for line in json_data:
    data_prompt = line['Problem']
    data_chosen = line['Solution']
    data_answer = line["Answer"]
    data.append({
        "query": data_prompt,
        "chosen": data_chosen,
        "answer": data_answer,
    })

# 保存为 JSON 文件
with open(json_file_path, "w") as json_file:
    json.dump(data, fp=json_file, indent=4, ensure_ascii=False)

print(f"Parquet 文件已成功转换为 JSON 并保存到 {json_file_path}")

Parquet 文件已成功转换为 JSON 并保存到 /home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/aime2024.json


# Split the data into 2 parts

In [3]:
import pandas as pd
import json
import re

# 读取 Parquet 文件
original_data_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/math_training/data/orca_10k_train-repeat.json"
output_path_1 = "/home/nfs02/laizj/experiment/uncertainty_analysis/math_training/data/orca_10k_train-repeat-first_half.json"
output_path_2 = "/home/nfs02/laizj/experiment/uncertainty_analysis/math_training/data/orca_10k_train-repeat-second_half.json"

with open(original_data_path, "r") as f:
    data = json.load(f)

with open(output_path_1, "w") as f:
    json.dump(data[:len(data)//2], f, indent=4, ensure_ascii=False)

with open(output_path_2, "w") as f:
    json.dump(data[len(data)//2:], f, indent=4, ensure_ascii=False)

# Few-shot data on math resoning

In [4]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/nfs05/laizj/model/models--Qwen--Qwen2.5-Math-7B-Instruct/snapshots/ef9926d75ab1d54532f6a30dd5e760355eb9aa4d")
input_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/gsm8k-test.json"
output_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/process_data/gsm8k-test-few_shot.json"

prompt = """### Instruction:
A rectangle has a length of 3 inches and a width of 6 inches. A square has a width of 5 inches. What is the difference in area between the two shapes?

### Response: 
Let’s think step by step.
The area of the rectangle is 3 x 6 = <<3*6=18>>18 sq in
The area of the square is 5 x 5 = <<5*5=25>>25 sq in
The difference is 25 - 18 = <<25-18=7>>7 sq in
#### 7

### Instruction:
{}

### Response: 
Let’s think step by step."""


with open (input_path, "r") as f:
    data = json.load(f)
    
for item in data:

    messages = [
        {"role": "system", "content": "Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step."},
        {"role": "user", "content": prompt.format(item["query"]) } # type: ignore
    ]
    
    item["prompt"] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True)

with open(output_path, "w") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)



/home/nfs01/anaconda3/envs/laizj-torch2.5-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Filte query data has abnormal length

In [ ]:
data_path_origin = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/gsm8k-train.json"
data_path_generate = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/disturbed_qa.jsonl"
output_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/gsm8k-train-disturbed.json"

prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step following the format \"The answer is\".

### Instruction:
{}

### Response: 
Let’s think step by step."""

with open(data_path_origin, 'r') as f:
    origin_data = json.load(f)
    
with open(data_path_generate, 'r') as f:
    generate_data = f.readlines()
    generate_data = [json.loads(x) for x in generate_data]
    
result = []
for generate_data_item in generate_data:
    origin_query = origin_data[generate_data_item['id']]['prompt']
    if abs(len(generate_data_item['query']) - len(origin_query)) / len(origin_query) < 0.2:
        # generate_data_item['prompt'] = prompt.format(generate_data_item['query'])
        generate_data_item['prompt'] = generate_data_item['query']
        result.append(generate_data_item)
        
with open(output_path, 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

# Data to generate template and python code

In [1]:


import json
from transformers import AutoTokenizer
import random

tokenizer = AutoTokenizer.from_pretrained("/home/nfs05/laizj/model/models--Qwen--Qwen2.5-Math-7B-Instruct/snapshots/ef9926d75ab1d54532f6a30dd5e760355eb9aa4d")
dataset_name="orca"
input_path = f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/raw_data/{dataset_name}-train.json"
output_path = f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/process_data/generate_template_and_code-{dataset_name}_20k_train.json"


instruction = r"""Task Description:
Create:
A variable substitution query template
A Python code generator that converts query responses to executable code. Don't add calculate in the comment.
Input Format:
Query: [Original query with specific values]
COT Response: [Chain-of-thought response explaining the solution]
Output Requirements:
Query Template:
Replace concrete values in the query with <variable_name> placeholders
Maintain original structure while making it generic
Python Code:
Start with variable definitions matching the template placeholders
Convert the COT logic to executable Python code
Include a final output statement
Preserve variable naming consistency with the template

Example:
### Query:
Find A that satisfies <coefficient>×A×A×A=<result>

### Response:
To find the value of A that satisfies the equation 32×A×A×A=42592, we can rewrite the equation as:
\(32A^3 = 42592\)
Now, we need to isolate A by dividing both sides of the equation by 32:
\(A^3 = \frac{42592}{32}\)
\(A^3 = 1331\)
Now, we take the cube root of both sides to solve for A:
\(A = \sqrt[3]{1331}\)
\(A = 11\)
Therefore, the answer is \(\boxed{11}\)

### Template:
Find A that satisfies <coefficient>×A×A×A=<result>

### Python Code:
```python
# Variable definitions
coefficient = 32
result = 42592

# Calculation
A_cubed = result / coefficient
A = A_cubed ** (1/3)

# Output
print(A)
```
"""

prompt = """
Instruction:
### Query:
{}

### Response:
{}

"""


with open(input_path, "r") as f:
    data = json.load(f)

random.shuffle(data)
data = data[:20000]

results = []
for idx, item in enumerate(data):
    
    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": instruction + prompt.format(item["query"], item["chosen"])},
    ]
    
    results.append({
        "id": idx,
        "query": item["query"],
        "prompt": tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True),
        "answer": item["answer"]
    })

with open(output_path, "w") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

/home/nfs01/anaconda3/envs/laizj-torch2.5-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
len("<|im_end|>\n<|im_start|>assistant\n")

64

# Data to generate COT with code

In [ ]:
from transformers import AutoTokenizer
import json

dataset="orca_20k_train-disturbed_1.0_16"
model_name="qwen8b"
tokenizer = AutoTokenizer.from_pretrained("/home/nfs05/laizj/model/models--Qwen--Qwen2.5-Math-7B-Instruct/snapshots/ef9926d75ab1d54532f6a30dd5e760355eb9aa4d")
input_path = f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/synthetic_data/{model_name}-generate_template_and_code-{dataset}.json"
output_path = f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/process_data/generate_cot_with_code-{model_name}-{dataset}.json"

prompt = r"""
Task Description:
Your task is to generate a chain-of-thought (COT) explanation that answers the user's question by reasoning through the logic implied in a provided Python script. Use the script to inform your explanation, but do not output or reproduce any code.

Input Format:
Query: A question involving specific values or conditions.
Python Code: A script that solves the query or provides a key computational procedure.

Output Requirements:
Start by interpreting the question clearly.
Reason through the problem step by step, using the Python code as a guide to inform your logic.
Refer to relevant steps in the code as part of your reasoning.
Do not output or reference the code in any form.
Explicitly state the final answer after the final step within \\boxed{}.
"""

instruction = """
### Query:
{}

### Python Code:
{}

### Resonse:
"""

with open(input_path, "r") as f:
    data = json.load(f)

results = []
for idx, item in enumerate(data):
    
    for inner_idx, inner_item in enumerate(item["disturbed"]):
        
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt + instruction.format(inner_item["new_query"], inner_item["new_code"])}
        ]
        
        results.append({
            "id": item.get("id", idx),
            "inner_id": inner_idx,
            "query": inner_item["new_query"],
            "prompt": tokenizer.apply_chat_template(
                        messages,
                        tokenize=False,
                        add_generation_prompt=True),
            "answer": inner_item["new_ans"]
        })

with open(output_path, "w") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

# Data to generate RFT

In [2]:
from transformers import AutoTokenizer
import json
import re
import nltk
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from tqdm import tqdm

nltk.data.path.append('/home/nfs03/laizj/model/nltk_data')
tokenizer = AutoTokenizer.from_pretrained("/home/nfs05/laizj/model/models--Qwen--Qwen2.5-Math-7B-Instruct/snapshots/ef9926d75ab1d54532f6a30dd5e760355eb9aa4d")

def extract_last_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", r"\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"\\boxed\{(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) == 0:
        res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        num_str = res[-1][0]
        return float(num_str)
    else:
        return 0.0

model_name="qwen7b_instruct"
dataset="qwen72b_instruct-generate_template_and_code-orca_10k_train-disturbed_1.0_16"
original_path=f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/synthetic_data/{dataset}.json"
input_path = f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/results/{model_name}-generate_cot_with_code-{dataset}.jsonl"
output_path = f"/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/process_data/generate_rft-{model_name}-generate_cot_with_code-{dataset}-4.json"   

with open(original_path, 'r') as f:
    original_data = json.load(f)

with open(input_path, 'r') as f:
    # data = json.load(f)
    data = []
    for line in f:
        data.append(json.loads(line))

processed_data = defaultdict(list)

average_cot_success_number = 0
average_cot_success_rate = 0

for item in tqdm(data):
    idx, inner_idx = map(int, item["idx"].split("_"))
    if idx not in processed_data:
        processed_data[idx].append({
            "question": original_data[idx]["query"],
            "chosen": original_data[idx]["response"],
            "answer": original_data[idx]["answer"],
        })
    for generated_text in item["model_output"]:
        if extract_last_num(generated_text) == item["new_ans"]:
            clauses = sent_tokenize(generated_text)
            
            for i in range(len(clauses) - 1, -1, -1):
                if extract_last_num(" ".join(clauses[:i])) != item["new_ans"] or "\\boxed{" in clauses[i]:
                    generated_text = generated_text[generated_text.rfind(clauses[i])].rstrip()
                    generated_text = " ".join(clauses[:i + 1])
                    break
            
            processed_data[idx].append({
                "question": item["new_query"],
                "chosen": generated_text,
                "answer": item["new_ans"],
            })
            
            break

result = []
for k, outer_item in processed_data.items():
    for _ in range(1):
        messages = [
            # {"role": "system", "content": "Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step within \\boxed{}."},
            {"role": "user", "content": outer_item[0]["question"]}
        ]
        
        result.append({
            "prompt": tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True),
            "instruction": outer_item[0]["question"],
            "output": outer_item[0]["chosen"],
            # "system": "Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step within \\boxed{}.",
            "input": "",
            "history": [],
            "answer": outer_item[0]["answer"],
            "original": True,
        })

    temp_result = []
    for item in outer_item[1:]:
        average_cot_success_number += 1
        
        messages = [
            # {"role": "system", "content": "Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step within \\boxed{}."},
            {"role": "user", "content": item["question"]}
        ]
        
        temp_result.append({
            "prompt": tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True),
            "instruction": item["question"],
            "output": item["chosen"],
            # "system": "Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step within \\boxed{}.",
            "input": "",
            "history": [],
            "answer": item["answer"],
        })
    
    if len(temp_result) >= 1:
        average_cot_success_rate += 1
        result.extend(temp_result[1:5])
    
with open(output_path, 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

print(f"数据个数:", len(result))
print(f"问题扰动获得正确COT的成功率：{average_cot_success_rate / len(processed_data) * 100:.2f}%")
print(f"平均每个问题的成功COT数量：{average_cot_success_number / len(processed_data):.2f}")

# 问题扰动获得正确COT的成功率：94.16%
# 平均每个问题的成功COT数量：3.18

# 11865-6080(one) 21588-6080 5928(test)

100%|██████████| 86065/86065 [00:21<00:00, 4086.29it/s]


数据个数: 25526
问题扰动获得正确COT的成功率：97.90%
平均每个问题的成功COT数量：10.48


In [4]:
str(torch.bfloat16)

'torch.bfloat16'